In [1]:
import torch, torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import time
from torchsummary import summary
import pathlib

import numpy as np
import matplotlib.pyplot as plt
import os

from PIL import Image

In [3]:
# Load the Data

data_dir = pathlib.Path('./data/tiny-imagenet-200')
image_count = len(list(data_dir.glob('**/*.JPEG')))
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
num_classes = len(CLASS_NAMES)
print('Discovered {} images in {} classes'.format(image_count, num_classes))

# Create the training data generator
batch_size = 32
im_height = 64
im_width = 64
num_epochs = 1

data_transforms_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_transforms_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load Data from folders
data = {
    'train': datasets.ImageFolder(root=data_dir / 'train', transform=data_transforms_train),
    'valid': datasets.ImageFolder(root=data_dir / 'val', transform=data_transforms_test),
    'test': datasets.ImageFolder(root=data_dir / 'test', transform=data_transforms_test)
}

# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in data['train'].class_to_idx.items()}
print(idx_to_class)

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size = len(data['train'])
valid_data_size = len(data['valid'])
test_data_size = len(data['test'])

# Create iterators for the Data loaded using DataLoader module
train_data_loader = DataLoader(data['train'], batch_size=batch_size, shuffle=True)
valid_data_loader = DataLoader(data['valid'], batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(data['test'], batch_size=batch_size, shuffle=True)

# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")

Discovered 129998 images in 200 classes
{0: 'n01443537', 1: 'n01629819', 2: 'n01641577', 3: 'n01644900', 4: 'n01698640', 5: 'n01742172', 6: 'n01768244', 7: 'n01770393', 8: 'n01774384', 9: 'n01774750', 10: 'n01784675', 11: 'n01855672', 12: 'n01882714', 13: 'n01910747', 14: 'n01917289', 15: 'n01944390', 16: 'n01945685', 17: 'n01950731', 18: 'n01983481', 19: 'n01984695', 20: 'n02002724', 21: 'n02056570', 22: 'n02058221', 23: 'n02074367', 24: 'n02085620', 25: 'n02094433', 26: 'n02099601', 27: 'n02099712', 28: 'n02106662', 29: 'n02113799', 30: 'n02123045', 31: 'n02123394', 32: 'n02124075', 33: 'n02125311', 34: 'n02129165', 35: 'n02132136', 36: 'n02165456', 37: 'n02190166', 38: 'n02206856', 39: 'n02226429', 40: 'n02231487', 41: 'n02233338', 42: 'n02236044', 43: 'n02268443', 44: 'n02279972', 45: 'n02281406', 46: 'n02321529', 47: 'n02364673', 48: 'n02395406', 49: 'n02403003', 50: 'n02410509', 51: 'n02415577', 52: 'n02423022', 53: 'n02437312', 54: 'n02480495', 55: 'n02481823', 56: 'n02486410', 

In [4]:
# Load pretrained ResNet50 Model
model = models.resnet50(pretrained=False)

# Change the final layer of ResNet50 Model for Transfer Learning
fc_inputs = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes), # Since 10 possible outputs
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)

# Convert model to be used on GPU
model = model.to('cuda:0')


best_model_path = "./model_best_resnet50.pth"
# model = torch.load(best_model_path)
state_dict = torch.load(best_model_path)['state_dict']
# model.load_state_dict({k: v for k, v in state_dict.items()})
model.load_state_dict({k[7:]: v for k, v in state_dict.items()})
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


In [13]:
%load_ext autoreload

import resnet_modified

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
%autoreload 2
import importlib
importlib.reload(resnet_modified)
model2 = resnet_modified.resnet50(pretrained=False, decay_factor=0.1)


# Change the final layer of ResNet50 Model for Transfer Learning
fc_inputs = model2.fc.in_features

model2.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes), # Since 10 possible outputs
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)


# Convert model to be used on GPU
model2 = model2.to('cuda:0')

model2.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): BundledBottleneck(
    (layers): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
         

In [61]:
import re
regex = '(layer..)(.*)'
new_state_dict = {}
for k, v in state_dict.items():
    k = k[7:]
    if re.match(regex, k):
        matches = re.findall(regex, k)[0]
        new_state_dict[matches[0] + 'layers.' + matches[1]] = v
    else:
        new_state_dict[k] = v
model2.load_state_dict(new_state_dict)

# inputs = ["layer3.layers.2.bn3.running_var", "layer3.layers.3.conv1.weight", "layer3.layers.3.bn1.weight", "layer3.layers.3.bn1.bias", "layer3.layers.3.bn1.running_mean", "layer3.layers.3.bn1.running_var", "layer3.layers.3.conv2.weight", "layer3.layers.3.bn2.weight", "layer3.layers.3.bn2.bias", "layer3.layers.3.bn2.running_mean", "layer3.layers.3.bn2.running_var", "layer3.layers.3.conv3.weight", "layer3.layers.3.bn3.weight", "layer3.layers.3.bn3.bias", "layer3.layers.3.bn3.running_mean", "layer3.layers.3.bn3.running_var", "layer3.layers.4.conv1.weight", "layer3.layers.4.bn1.weight"]
# for x in inputs:
#     print(x, ''.join(re.findall(regex, x)[0]))

# model2.load_state_dict({k[7:]: v for k, v in state_dict.items()})

<All keys matched successfully>

In [47]:
import train_model_within_programs

In [48]:
importlib.reload(train_model_within_programs)
args = ["./data/tiny-imagenet-200/", "--epochs", "10", "--filename", "resnet50_fh_v1", "--evaluate"]
train_model_within_programs.run(args, model, data['train'], data['valid'])

Epoch: 10 batch_size: 32
Test: [  0/313]	Time  0.113 ( 0.113)	Loss 9.4334e+00 (9.4334e+00)	Acc@1  12.50 ( 12.50)	Acc@5  40.62 ( 40.62)
Test: [ 10/313]	Time  0.041 ( 0.051)	Loss 1.2515e+01 (1.4714e+01)	Acc@1   0.00 (  3.41)	Acc@5   6.25 (  9.66)
Test: [ 20/313]	Time  0.035 ( 0.045)	Loss 9.0967e+00 (1.4588e+01)	Acc@1  12.50 (  3.87)	Acc@5  46.88 ( 11.31)
Test: [ 30/313]	Time  0.036 ( 0.042)	Loss 1.5205e+01 (1.3458e+01)	Acc@1   0.00 (  6.85)	Acc@5   3.12 ( 15.42)
Test: [ 40/313]	Time  0.036 ( 0.041)	Loss 2.0747e+01 (1.3407e+01)	Acc@1   3.12 (  8.46)	Acc@5  15.62 ( 17.30)
Test: [ 50/313]	Time  0.036 ( 0.040)	Loss 1.3747e+01 (1.4258e+01)	Acc@1   0.00 (  7.11)	Acc@5   0.00 ( 14.95)
Test: [ 60/313]	Time  0.037 ( 0.039)	Loss 1.2411e+01 (1.3951e+01)	Acc@1   3.12 (  7.33)	Acc@5  25.00 ( 15.78)
Test: [ 70/313]	Time  0.036 ( 0.039)	Loss 1.0274e+01 (1.3374e+01)	Acc@1   3.12 (  7.13)	Acc@5  21.88 ( 17.52)
Test: [ 80/313]	Time  0.036 ( 0.039)	Loss 1.4459e+01 (1.3315e+01)	Acc@1   0.00 (  6.71)	Acc@5  

In [46]:
importlib.reload(train_model_within_programs)
args = ["./data/tiny-imagenet-200/", "--epochs", "10", "--filename", "resnet50_v1", "--evaluate"]
train_model_within_programs.run(args, model, data['train'], data['valid'])

Epoch: 10 batch_size: 32


RuntimeError: DataLoader worker (pid(s) 2793) exited unexpectedly

In [7]:
import importlib
import train_model_within_programs
# importlib.reload(train_model_within_programs)
args = ["./data/tiny-imagenet-200/", "--epochs", "10", "--filename", "resnet50_fh_v1", "--evaluate"]
train_data, val_data = train_model_within_programs.run(args, model, data['train'], data['valid'])

Epoch: 10 batch_size: 32
Test: [  0/313]	Time  1.873 ( 1.873)	Loss 7.2961e+00 (7.2961e+00)	Acc@1  15.62 ( 15.62)	Acc@5  34.38 ( 34.38)
Test: [ 10/313]	Time  0.036 ( 0.203)	Loss 9.3720e+00 (9.3836e+00)	Acc@1   0.00 (  1.99)	Acc@5   0.00 (  7.39)
Test: [ 20/313]	Time  0.032 ( 0.123)	Loss 8.2974e+00 (9.3338e+00)	Acc@1  18.75 (  2.98)	Acc@5  31.25 (  9.67)
Test: [ 30/313]	Time  0.038 ( 0.095)	Loss 1.1526e+01 (9.0059e+00)	Acc@1   0.00 (  4.33)	Acc@5   3.12 ( 13.61)
Test: [ 40/313]	Time  0.036 ( 0.081)	Loss 1.2945e+01 (9.3206e+00)	Acc@1  12.50 (  5.87)	Acc@5  25.00 ( 14.79)
Test: [ 50/313]	Time  0.036 ( 0.072)	Loss 1.0580e+01 (9.9166e+00)	Acc@1   0.00 (  5.21)	Acc@5   6.25 ( 13.30)
Test: [ 60/313]	Time  0.035 ( 0.066)	Loss 5.3992e+00 (9.7078e+00)	Acc@1   6.25 (  5.43)	Acc@5  46.88 ( 14.75)
Test: [ 70/313]	Time  0.038 ( 0.062)	Loss 8.0499e+00 (9.3959e+00)	Acc@1   3.12 (  5.41)	Acc@5  21.88 ( 15.49)
Test: [ 80/313]	Time  0.035 ( 0.059)	Loss 1.1034e+01 (9.5959e+00)	Acc@1   0.00 (  4.90)	Acc@5  

In [56]:
val_data

[26781.302511305817,
 tensor(1.2501, device='cuda:0'),
 tensor(5.1805, device='cuda:0')]

In [66]:
%autoreload 2
import importlib
importlib.reload(resnet_modified)
model2 = resnet_modified.resnet50(pretrained=True, decay_factor=0.1)


# Change the final layer of ResNet50 Model for Transfer Learning
fc_inputs = model2.fc.in_features

model2.fc = nn.Sequential(
    nn.Linear(fc_inputs, 256),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, num_classes), # Since 10 possible outputs
    nn.LogSoftmax(dim=1) # For using NLLLoss()
)


# Convert model to be used on GPU
model2 = model2.to('cuda:0')

model2.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): BundledBottleneck(
    (layers): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
         

In [67]:
importlib.reload(train_model_within_programs)
args = ["./data/tiny-imagenet-200/", "--epochs", "3", "--filename", "resnet50_fh_v1", "--print-freq", "50"]
train_model_within_programs.run(args, model2, data['train'], data['valid'])

Epoch: 3 batch_size: 32
Epoch:0 top1: Acc@1   0.00 (  0.00) top5: Acc@5   0.00 (  0.00)


OSError: [Errno 12] Cannot allocate memory